## インポート

In [52]:
from config import NUMBERS, NUMBERS3, NUMBERS4, NUMBERS4_RESULT
import sys
import re
import difflib
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### NUMBERS4 で予測する

In [20]:
X = []
Y = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0

key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        # if len(list(set(_y) & set(_v))) >= 3:
        if len(list(set(_y) & set(_v))) >= 4 and _z == "先負":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_format)
    # Y.append(_y)
    # X.append(_x)
    pass
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)
_sum = sum([key_0, key_1, key_2, key_3, key_4, key_5])

print(_format1)

前回:0個, 前々回:0個, 前回曜日:1個, 前々回曜日:0個, 前回六曜:0個, 前々回六曜:1個


#### 1. 前回のナンバーを特徴量とする

In [32]:
X = []
Y = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4[_i][_time][0]
    if len(list(set(_y) & set(_x))) >= 3:
        #print("{0} -> {1}:{2}".format(_time, _y, _x))
        Y.append(_y)
        X.append([_x])

#### 2. 前回の六曜のナンバーを特徴量とする

In [31]:
X = []
Y = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4[_i][_time][4]
    if len(list(set(_y) & set(_x))) >= 3:
        # print("{0} -> {1}:{2}".format(_time, _y, _x))
        Y.append(_y)
        X.append([_x])

#### 3. 前回のナンバーと前回の六曜のナンバーを特徴量とする

In [35]:
X = []
Y = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x1 = NUMBERS4[_i][_time][4]
    _x2 = NUMBERS4[_i][_time][0]
    if len(list(set(_y) & set(_x1))) >= 3 or len(list(set(_y) & set(_x2))) >= 3:
        Y.append(_y)
        X.append([_x1, _x2])
        # print("{0} -> {1}:{2}".format(_time, _y, _x))

#### 4. 前回曜日のナンバーと前回六曜のナンバーを特徴量とする


In [4]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x1 = NUMBERS4[_i][_time][2]
    _x2 = NUMBERS4[_i][_time][4]
    _z = NUMBERS[_i][4]
    # if len(list(set(_y) & set(_x1))) >= 3 or len(list(set(_y) & set(_x2))) >= 3:
    if len(list(set(_y) & set(_x1))) >= 3 or len(list(set(_y) & set(_x2))) >= 3 and _z == "先負":
        Y.append(_y)
        X.append([_x1, _x2])
        TIME.append(_time)
        pass
    # Y.append(_y)
    # X.append([_x1, _x2])
    # TIME.append(_time)

#### 5. 前回曜日のナンバーと前々回六曜のナンバーを特徴量とする

In [36]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x1 = NUMBERS4[_i][_time][2]
    _x2 = NUMBERS4[_i][_time][5]
    _z = NUMBERS[_i][4]
    if _z == "先負":
        Y.append(_y)
        X.append([_x1, _x2])
        TIME.append(_time)
        pass
    # Y.append(_y)
    # X.append([_x1, _x2])
    # TIME.append(_time)

#### 6. 抽選数を特徴量とする

In [2]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4_RESULT[_i][_time]
    Y.append(_y)
    X.append(_x)
    TIME.append(_time)

In [37]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [38]:
len(y)

192

In [39]:
len(TIME)

192

In [40]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [41]:
px = x
py = model.predict(px)

In [42]:
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[3]) & set(_py))) >= 3:
                print(TIME[_i], _v[3], py[_i])
            # print(TIME[_i], _v[3], py[_i])

4812 3627 4763.819453447611
4808 2348 5428.835650661027
4743 7614 4916.249964858621
4163 7145 5184.442591167411
4081 5096 5089.012516088525
4019 0286 5082.041749687637
3888 5946 5269.846155413612


## 決定係数

In [43]:
r2_score(y, py)

0.005129711967710615

In [44]:
model.score(x, y)

0.005129711967710615

In [45]:
model.coef_[0]

0.0512778258656091

## 予測

In [46]:
pkl_file = "numbers4_20180618_6.pkl"

#### 前回ナンバーと前回曜日ナンバー

#### 前回曜日ナンバーと前々回六曜ナンバー

In [47]:
# data = [float("5935"), float("3636")]
data = [float("3076"), float("5382")]

#### 抽選数

In [13]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x0 = NUMBERS4_RESULT[_i][_time][0]
    _x1 = NUMBERS4_RESULT[_i][_time][1]
    _x2 = NUMBERS4_RESULT[_i][_time][2]
    _x3 = NUMBERS4_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

# 20180618
# data = [float("1"), float("1"), float("1"), float("1")]
data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

22 270 71 1185


In [48]:
joblib.dump(model, pkl_file)

['numbers4_20180618_6.pkl']

In [49]:
clf = joblib.load(pkl_file)

In [50]:
processor = clf.predict([data])

In [51]:
processor

array([5075.67352375])